In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd 
import datetime 

import matplotlib.pyplot as plt 
import seaborn as sns 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_set = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test_set = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
items_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_categories_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
submit_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
train_set

In [ ]:
#Negatif ürün satırlarının kaldırılması
train_set = train_set[train_set.item_price > 0].reset_index(drop = True)
train_set.loc[train_set.item_cnt_day < 1, "item_cnt_day"] = 0

In [ ]:
train_set.date=train_set.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
train_set['year'] = pd.to_datetime(train_set['date']).dt.strftime('%Y')
train_set['month'] = pd.to_datetime(train_set['date']).dt.strftime('%m')

In [ ]:
grouped = pd.DataFrame(train_set.groupby(['year','month'])['item_cnt_day'].sum().reset_index())
sns.pointplot(x='month', y='item_cnt_day', hue='year', data=grouped)

In [ ]:
ts=train_set.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.title('Toplam Satış')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts)

In [ ]:
#Train veri setindeki item_cnt_day ve item_price ayrık veri gözlemleme

plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train_set.item_cnt_day)

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(train_set.item_price.min(), train_set.item_price.max()*1.1)
sns.boxplot(x=train_set.item_price)

In [ ]:
#Aykırı değerlerin kaldırılması
train_set = train_set[(train_set.item_price < 100000  ) & (train_set.item_cnt_day < 800)]

In [ ]:
# Test veri setinde olup Train veri setinde olamayan veri sayıları bulma
len(test_set), len(list(set(test_set.item_id))), len(list(set(test_set.item_id) - set(test_set.item_id).intersection(set(train_set.item_id)))) 

In [ ]:
monthly_sales=train_set.groupby(["date_block_num","shop_id","item_id"])[
    "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})
monthly_sales

In [ ]:
x=items_data.groupby(['item_category_id']).count().sort_values(by='item_id',ascending=False)
x=x.reset_index()
x

In [ ]:
plt.figure(figsize=(3,32))
ax= sns.barplot(x.item_category_id, x.item_id, alpha=0.8)
plt.title("Items per Category")
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.show()

In [ ]:
items_data["item_name_size"] = items_data["item_name"].map(len)
items_data = items_data.drop("item_name", axis = 1)

item_categories_data["category_name_size"] = item_categories_data["item_category_name"].map(len)
item_categories_data = item_categories_data.drop(["item_category_id", "item_category_name"], axis = 1)

items_data["category_name_size"] = items_data["item_category_id"].map(lambda x : item_categories_data.iloc[x, 0])

items_data = items_data.set_index('item_id')

display(items_data)

In [ ]:
train_set = train_set.set_index('item_id')
train_set = train_set.join(items_data)
train_set = train_set.reset_index()

In [ ]:
train_set = train_set.set_index('shop_id')
train_set = train_set.join(shops_data)
train_set = train_set.reset_index()

In [ ]:
train_set

In [ ]:
grouped = pd.DataFrame(train_set.groupby(['shop_id', 'date_block_num','item_id'])['item_cnt_day'].sum().reset_index())
total_item_cnt_mounth = grouped.groupby('date_block_num')['item_cnt_day'].sum()

In [ ]:
total_item_cnt_mounth = grouped.groupby('date_block_num')['item_cnt_day'].sum()

In [ ]:
total_item_cnt_mounth

In [ ]:
total_item_cnt_mounth_np = total_item_cnt_mounth.to_numpy()
total_item_cnt_mounth_np
mounths = np.arange(34)

In [ ]:
corrmat = train_set.corr()
k = 10 
cols = corrmat.nlargest(k, 'item_cnt_day')['item_cnt_day'].index
cm = np.corrcoef(train_set[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mounths,total_item_cnt_mounth_np, test_size = 2/3, random_state = 123, shuffle=2)

In [ ]:
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_train,y_train)

In [ ]:
from sklearn import tree
model_DTC = tree.DecisionTreeClassifier(criterion='gini') 

In [ ]:
model_DTC.fit(X_train, y_train)
model_DTC.score(X_train,y_train)

In [ ]:
cols = ['year','date_block_num','month','item_price']
X = train_set[cols] 
y = train_set.item_cnt_day 

In [ ]:
model.fit(X,y)

In [ ]:
model_DTC.fit(X, y)

In [ ]:
print("LinearRegressionScore",model.score(X, y))
resultLinear = model.predict(X)

In [ ]:
print("DTCscore",model_DTC.score(X, y))
result = model_DTC.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, result)
print(accuracy)